# **Spectral Index Evaluation of Land Use Compliance with the PBOT in La Ceja**
This project aims to analyze and quantify the urban expansion in La Ceja, a municipality located in Eastern Antioquia (Colombia), between the years 2018 and 2025. Using Sentinel-2 multispectral satellite imagery, we calculate spectral indices such as NDVI (Normalized Difference Vegetation Index) and NDBI (Normalized Difference Built-up Index) to detect changes in vegetation cover and built-up areas.

These spatial transformations are then compared with the urban growth zones defined in the Municipal Land Use Plan (PBOT) established by Agreement 001 of 2018. The overlay between remote sensing analysis and official planning boundaries allows us to assess whether recent urban development has occurred within legally designated areas or has exceeded regulated expansion zones.

This approach provides technical evidence to support sustainable territorial management, monitor compliance with planning regulations, and inform future decision-making by local authorities. The project also explores the effectiveness of remote sensing as a tool for land-use monitoring in rapidly growing intermediate municipalities.


## Libraries and necessary packs

In [9]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import pandas as pd
import seaborn as sns
import geopandas as gpd
from pathlib import Path
import glob
import os
from tqdm import tqdm
from IPython.display import display

## Upload and stack verification

Code to verify the content of archives ".tif", it makes the band decomposition, index and description of each one

In [ ]:
import rasterio
import os

stack_dir = "./carpeta_stacks" # Stacks directory
for filename in os.listdir(stack_dir):
    if filename.endswith(".tif"):
        file_path = os.path.join(stack_dir, filename)
        with rasterio.open(file_path) as src_file:
            print(f"Archivo: {filename}")
            print("Número de bandas:", src_file.count)
            print("Índices de banda disponibles:", src_file.indexes)
            desc = src_file.descriptions
            if desc is None:
                print("No hay descripciones de banda asociadas.")
            else:
                for idx, d in enumerate(desc, start=1):
                    print(f"Banda {idx}: {d}")
            print("-" * 40) # Separator for clarity


Archivo: 2018-08-07_stack3.tif
Número de bandas: 3
Índices de banda disponibles: (1, 2, 3)
Banda 1: NDVI
Banda 2: NDBI
Banda 3: UI
----------------------------------------
Archivo: 2019-01-04_stack3.tif
Número de bandas: 3
Índices de banda disponibles: (1, 2, 3)
Banda 1: NDVI
Banda 2: NDBI
Banda 3: UI
----------------------------------------
Archivo: 2019-07-18_stack3.tif
Número de bandas: 3
Índices de banda disponibles: (1, 2, 3)
Banda 1: NDVI
Banda 2: NDBI
Banda 3: UI
----------------------------------------
Archivo: 2019-08-27_stack3.tif
Número de bandas: 3
Índices de banda disponibles: (1, 2, 3)
Banda 1: NDVI
Banda 2: NDBI
Banda 3: UI
----------------------------------------
Archivo: 2020-01-09_stack3.tif
Número de bandas: 3
Índices de banda disponibles: (1, 2, 3)
Banda 1: NDVI
Banda 2: NDBI
Banda 3: UI
----------------------------------------
Archivo: 2020-02-18_stack3.tif
Número de bandas: 3
Índices de banda disponibles: (1, 2, 3)
Banda 1: NDVI
Banda 2: NDBI
Banda 3: UI
---------

This section loads our training points from the GeoPackage "muestras_training.gpkg" obtained with QGIS previously ("points_training" layer), finds all multiband GeoTIFF “stacks” (each containing NDVI, NDBI and UI bands), and samples those three index values at every training point location.  

- **Load**: Read the "points_training" layer with GeoPandas.  
- **List**: Use "glob" to locate all stack files matching the given pattern.  
- **Sample**: For each stack, open it with Rasterio and sample the NDVI, NDBI, and UI bands at each point’s (x, y).  
- **Assemble**: Build a single pandas "DataFrame" ("df_samples") with columns:
  - `raster_date`: scene date (parsed from filename)  
  - `point_date`: original sampling date  
  - `label`: land-cover class  
  - `x`, `y`: point coordinates  
  - `NDVI`, `NDBI`, `UI`: extracted index values  
- **Inspect**: Print dimensions and display the first few rows to verify the results.


In [12]:
# Parámetros — ajusta esta ruta a donde estén tus stacks
gpkg_path = './carpeta_stacks/muestras_training.gpkg'
layer_name = 'points_training'
stacks_path_pattern = './carpeta_stacks/*.tif'  # ← Cambia '/mnt/data/indices/' por tu carpeta

# 1. Cargar los puntos de entrenamiento
gdf = gpd.read_file(gpkg_path, layer=layer_name)
coords = [(pt.x, pt.y) for pt in gdf.geometry]

# 2. Listar archivos de stacks multibanda
raster_files = sorted(glob.glob(stacks_path_pattern))
print(f"Encontrados {len(raster_files)} archivos de stack:")
for fp in raster_files:
    print(" •", fp)

# 3. Extraer valores
records = []
for rf in tqdm(raster_files, desc="Procesando stacks"):
    date_str = os.path.splitext(os.path.basename(rf))[0]
    with rasterio.open(rf) as src:
        samples = list(src.sample(coords))
        for (ndvi, ndbi, ui), row in zip(samples, gdf.itertuples()):
            records.append({
                'raster_date': date_str,
                'point_date': row.date,
                'label': row.label,
                'x': row.geometry.x,
                'y': row.geometry.y,
                'NDVI': ndvi,
                'NDBI': ndbi,
                'UI': ui
            })

# 4. Construir DataFrame
df_samples = pd.DataFrame.from_records(records)

# 5. Visualización simple en Jupyter
print("\nDimensiones del DataFrame:", df_samples.shape)
print("Columnas:", df_samples.columns.tolist())
display(df_samples.head(10))

Encontrados 7 archivos de stack:
 • ./carpeta_stacks\2018-08-07_stack3.tif
 • ./carpeta_stacks\2019-01-04_stack3.tif
 • ./carpeta_stacks\2019-07-18_stack3.tif
 • ./carpeta_stacks\2019-08-27_stack3.tif
 • ./carpeta_stacks\2020-01-09_stack3.tif
 • ./carpeta_stacks\2020-02-18_stack3.tif
 • ./carpeta_stacks\2023-08-26_stack3.tif


Procesando stacks: 100%|██████████| 7/7 [00:01<00:00,  4.37it/s]


Dimensiones del DataFrame: (52920, 8)
Columnas: ['raster_date', 'point_date', 'label', 'x', 'y', 'NDVI', 'NDBI', 'UI']


,raster_date,point_date,label,x,y,NDVI,NDBI,UI
0,2018-08-07_stack3,2019-07-18,1,-75.428738,6.030611,0.113764,0.151539,0.142384
1,2018-08-07_stack3,2019-07-18,1,-75.431324,6.032770,0.144415,0.165567,0.068236
2,2018-08-07_stack3,2019-07-18,1,-75.431388,6.036990,0.211147,0.009404,-0.914724
3,2018-08-07_stack3,2019-07-18,1,-75.429235,6.033183,0.088880,0.204575,0.394252
4,2018-08-07_stack3,2019-07-18,1,-75.437057,6.025213,0.071481,0.167425,0.401596
5,2018-08-07_stack3,2019-07-18,1,-75.428785,6.031378,0.146936,0.229016,0.218325
6,2018-08-07_stack3,2019-07-18,1,-75.432873,6.027233,0.023427,0.117285,0.667015
7,2018-08-07_stack3,2019-07-18,1,-75.442700,6.020924,0.229805,-0.004953,-1.000000
8,2018-08-07_stack3,2019-07-18,1,-75.435093,6.033707,0.280773,0.156348,-0.284644
9,2018-08-07_stack3,2019-07-18,1,-75.438681,6.020314,0.124764,0.187344,0.200508
